Dijkstra's Shortest Path Algorithm
-------------------------------------------------

First define the test map as an array. Each entry in the array represents a grid cell on the map.

In [17]:
import numpy as np

test_map1 = np.array([
              [1, 1, 1, 1, 1, 1, 1, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 0, 0, 0, 0, 0, 0, 1],
              [1, 1, 1, 1, 1, 1, 1, 1]])
occupancy_map = test_map1

Define the spacing in the x (horizontal) direction between adjacent columns and the y (vertical) direction between adjacent rows. The positive x direction is to the right and the positive y direction is **downward**.

In [18]:
x_spacing1 = 0.13
y_spacing1 = 0.2
x_spacing = x_spacing1
y_spacing = y_spacing1

Define the start and goal positions.

In [19]:
start1 = np.array([[0.3], [0.3], [0]])
goal1 = np.array([[0.6], [1], [0]])
start = start1
goal = goal1

Keep track of the state of each grid cell by populating a new array. A value of 1 represents a free cell and a value of 2 represents an obstacle cell.

In [20]:
state_map = np.zeros(occupancy_map.shape) # state_map.shape = (8, 8)
state_map[occupancy_map == 0] = 1
state_map[occupancy_map == 1] = 2
state_map

array([[2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 1., 1., 1., 1., 1., 1., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.]])

Next we need to determine the node index of the starting position in both the x and y directions. A node is defined to be at the center of a grid cell. However, it is possible (and most likely the case) that the starting position is not exactly lined up with node and we need to find out which one of the four cases is true:

1. Start position is between two nodes in the x-direction.
2. Start position is between two nodes in the y-direction.
3. Start position is between two nodes in the x and y directions.
4. Start position is exactly on top of a node.

Let's define a grid in the x and y directions to check for each of the cases. Array entries represent the position of the left edge of the grid cell (not the node).

In [58]:
x_grid = np.arange(state_map.shape[1])*x_spacing
y_grid = np.arange(state_map.shape[0])*y_spacing
print("x_grid = " + str(x_grid))
print("y_grid = " + str(y_grid))

x_grid = [0.   0.13 0.26 0.39 0.52 0.65 0.78 0.91]
y_grid = [0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8]


Check if starting position is in between two nodes in the x-direction. This means the starting x-coordinate is on the edge of a grid cell.

In [46]:
# tol = 1e-6
startBtw2xNodes = False
startBtw2yNodes = False
if np.mod(start[0], x_spacing) == 0: # starting position exactly on edge of grid cell
    jStart = np.asscalar(np.searchsorted(x_grid, start[0])) # find index in x_grid that corresponds to starting x-coordinate
    startBtw2xNodes = True
else:
    jStart = np.asscalar(np.searchsorted(x_grid, start[0])) - 1 # find index in x_grid to the left of starting x-coordinate
    
print("startBtw2xNodes = " + str(startBtw2xNodes)) # start = (0.3, 0.3) so should be false
print("jStart = " + str(jStart))

startBtw2xNodes = False
jStart = 2


Repeat the same procedure for y-direction. Check if starting position is in between two nodes in the y-direction. This means the starting y-coordinate is on the edge of a grid cell.

In [47]:
if np.mod(start[1], y_spacing) == 0: # starting position exactly on edge of grid cell
    iStart = np.asscalar(np.searchsorted(y_grid, start[1])) # find index in x_grid that corresponds to starting x-coordinate
    startBtw2yNodes = True
else:
    iStart = np.asscalar(np.searchsorted(y_grid, start[1])) - 1 # find index in x_grid to the left of starting x-coordinate
    
print("startBtw2yNodes = " + str(startBtw2yNodes)) # start = (0.3, 0.3) so should be false
print("iStart = " + str(iStart))

startBtw2yNodes = False
iStart = 1


Store all of the possible starting nodes in a list. If the starting position is exactly in between two nodes in the x and/or y-directions then this increases the number of possible starting nodes. If the starting position is exactly on a node, then the starting node is the starting position.

In [51]:
if startBtw2xNodes and startBtw2yNodes:
    possibleStartNodes = [(iStart,jStart),(iStart,jStart-1),(iStart-1,jStart),(iStart-1,jStart-1)]
elif startBtw2xNodes:
    possibleStartNodes = [(iStart,jStart),(iStart,jStart-1)]
elif startBtw2yNodes:
    possibleStartNodes = [(iStart,jStart),(iStart-1,jStart)]
else:
    possibleStartNodes = [(iStart,jStart)]
    
print("possibleStartNodes =")
print('\n'.join(map(str, possibleStartNodes)))

possibleStartNodes =
(1, 2)
